In [1]:
#Import required Image library
from PIL import Image, ImageFilter

#Open existing image
OriImage = Image.open('clear.jpg')
OriImage.show()

blurImage = OriImage.filter(ImageFilter.BLUR)
blurImage.show()
#Save blurImage
blurImage.save('clear_blurred.jpg')



In [2]:
import os
import glob
from PIL import Image, ImageFilter

# Set the paths for the input and output directories
input_dir = 'without_COTS/'
output_dir = 'output/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Set the image extension to process
image_ext = "*.jpg"

# Get a list of all the image file paths in the input directory
image_paths = glob.glob(os.path.join(input_dir, image_ext))

# Loop through each image file and apply the blur effect
for image_path in image_paths:
    # Open the image file
    image = Image.open(image_path)

    # Apply the blur effect
    blur_image = image.filter(ImageFilter.BLUR)

    # Construct the output file path
    output_path = os.path.join(output_dir, os.path.basename(image_path))

    # Save the blurred image to the output directory
    blur_image.save(output_path)


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os

# Load the input and target images
input_images = []
target_images = []

# Load the images from the 'input' and 'target' directories
for file in os.listdir('output'):
    input_image = cv2.imread(os.path.join('output', file))
    target_image = cv2.imread(os.path.join('without_COTS', file))

    input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    target_image = cv2.cvtColor(target_image, cv2.COLOR_BGR2RGB)

    input_image = input_image / 255.0
    target_image = target_image / 255.0

    input_images.append(input_image)
    target_images.append(target_image)

# Convert the input and target image lists to numpy arrays
input_images = np.array(input_images)
target_images = np.array(target_images)

# Define the CNN architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_images.shape[1:]),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(3, (3, 3), activation='linear', padding='same')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the input and target images
model.fit(input_images, target_images, epochs=10)

# Use the trained model to deblur a new image
new_image = cv2.imread('cots pilot project-20230221T054935Z-001/cots pilot project/testing.jpg')
new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB) / 255.0
new_image = new_image.reshape((1,) + new_image.shape)
deblurred_image = model.predict(new_image)[0]

# Save the deblurred image
deblurred_image = cv2.cvtColor(np.uint8(deblurred_image * 255.0), cv2.COLOR_RGB2BGR)
cv2.imwrite('deblurred_image.png', deblurred_image)


Epoch 1/10
